At the begining let's install scraping package and parser.
Import nessesary libraries.

In [1]:
!pip install beautifulsoup4
!pip install lxml

In [2]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd

Create csv file and start the scraping/parsing process.

In [3]:
csv_wiki_table = open('csv_wiki_table.csv', 'w') #open the csv file for editing 
csv_writer = csv.writer(csv_wiki_table)
csv_writer.writerow(['PostalCode', 'Borough', 'Neighborhood']) #create head row

wiki_page = requests.get(' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(wiki_page, 'lxml') #parse the page

 Use print(soup.prettify()) to show the html file in structured view.

Cut the needed table from the file and write the needed data to the csv file.

In [4]:
wiki_table = soup.find('table', class_ = 'wikitable sortable')

In [5]:
for row in wiki_table.find_all('tr'):
    list_item = []
    for item in row.find_all('td'):
        list_item.append(item.text)
    csv_writer.writerow(list_item)


In [6]:
csv_wiki_table.close()

Import csv file to pandas and clear the data for futher analysis.

In [7]:
pd_wiki = pd.read_csv('csv_wiki_table.csv')
pd_wiki['Neighborhood'] = pd_wiki['Neighborhood'].map(lambda x: x.replace('\n', '')) #remove /n
print(pd_wiki.head())

  PostalCode           Borough      Neighborhood
0        M1A      Not assigned      Not assigned
1        M2A      Not assigned      Not assigned
2        M3A        North York         Parkwoods
3        M4A        North York  Victoria Village
4        M5A  Downtown Toronto      Harbourfront


In [8]:
pd_wiki = pd_wiki[pd_wiki['Borough'] != 'Not assigned'] #remove NA Boroughs
pd_wiki.loc[pd_wiki['Neighborhood'] == 'Not assigned', "Neighborhood"] = pd_wiki['Borough'] #replace NA Neighb
wiki_df = pd_wiki.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(','.join).reset_index()
wiki_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Export pandas dataframe to csv file to use it in another notebook.

In [9]:
wiki_df.to_csv('wiki_df.csv', index = False)

In [10]:
wiki_df.shape

(103, 3)